# Pre-processing pipeline

In [1]:
import mne
import numpy
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import concurrent.futures
import pywt
import scipy as sp
from sklearn.model_selection import StratifiedGroupKFold
import re

### configuration setup

In [2]:
mne.set_log_level('WARNING')

RAW_PATH = '/home/nis/Git/tuh-eeg-seizure-detection/data/raw'
OUTPUT_PATH = '/home/nis/Git/tuh-eeg-seizure-detection/data/processed_fast'

SAMPLING_FREQ = 250
WINDOW_LENGTH = 30
OVERLAP = 10 
CONFIGURATIONS = ["01_tcp_ar"]
CHANNELS = ["EEG FP1-REF", "EEG FP2-REF", "EEG F7-REF", "EEG F3-REF", "EEG F4-REF", "EEG F8-REF", "EEG T3-REF", "EEG C3-REF", "EEG C4-REF", "EEG T4-REF", "EEG T5-REF", "EEG P3-REF", "EEG P4-REF", "EEG T6-REF", "EEG O1-REF", "EEG O2-REF", "EEG CZ-REF", "EEG A1-REF", "EEG A2-REF"]

In [3]:
def split_channels_to_hemispheres(channels: list):
    left_hemisphere = []
    right_hemisphere = []
    
    for channel in channels:
        channel_number = re.search(r'\d+', channel)
        if channel_number is None:
            continue
        
        if int(channel_number.group()) % 2 == 0:
            right_hemisphere.append(channel)
        else:
            left_hemisphere.append(channel)
    
    return left_hemisphere, right_hemisphere

LEFT_HEMISPHERE, RIGHT_HEMISPHERE = split_channels_to_hemispheres(CHANNELS) 

### load windows

In [4]:
def extract_events_from_annotations(annotation_file):
    with open(annotation_file, "r") as f:
        annotations = f.readlines()
        events = annotations[6:] 
        
        data = []
        for event in events:
            parts = event.split(",")
            
            start = float(parts[1])
            stop = float(parts[2])
            label = parts[3]
            
            data.append({
                "label": label,
                "start": start,
                "stop": stop,
            })
            
    return pd.DataFrame(data)

In [22]:
def load_windows():
    cols = ["set", "patient_id", "session_id", "configuration", "recording_id", "recording_path", "event_index", "start", "stop", "label"]
    data = []
    
    edf_path = os.path.join(RAW_PATH, "edf")
    
    for root, _, files in os.walk(edf_path):
        for file in files:
            if not file.endswith(".edf"):
                continue
            
            rel_path = os.path.relpath(root, edf_path)
            parts = rel_path.split("/")
            
            if len(parts) != 4:
                continue
        
            set_name, patient_id, session_id, configuration = parts
            
            if configuration not in CONFIGURATIONS:
                continue
        
            recording_path = os.path.join(root, file)
            recording_id = file.replace(".edf", "").split("_")[-1]
            annotation_path = recording_path.replace(".edf", ".csv_bi")
            
            if not os.path.exists(recording_path) or not os.path.exists(annotation_path):
                continue
            
            events = extract_events_from_annotations(annotation_path)
            
            for i, event in events.iterrows():
                start, stop, label = event.loc[["start", "stop", "label"]]
                duration = stop - start

                if duration < WINDOW_LENGTH:
                    continue

                while start + WINDOW_LENGTH < stop:
                    data.append({
                        "set": set_name,
                        "patient_id": patient_id,
                        "session_id": session_id,
                        "configuration": configuration,
                        "recording_id": recording_id,
                        "recording_path": recording_path,
                        "event_index": i,
                        "start": start,
                        "stop": start + WINDOW_LENGTH,
                        "label": label,
                    })
                    
                    start += WINDOW_LENGTH - OVERLAP

    return pd.DataFrame(data, columns=cols)

windows = load_windows()
windows 

,set,patient_id,session_id,configuration,recording_id,recording_path,event_index,start,stop,label
0,train,aaaaates,s007_2015,01_tcp_ar,t002,/home/nis/Git/tuh-eeg-seizure-detection/data/r...,3,176.0310,206.0310,seiz
1,train,aaaaates,s007_2015,01_tcp_ar,t002,/home/nis/Git/tuh-eeg-seizure-detection/data/r...,3,196.0310,226.0310,seiz
2,train,aaaaates,s007_2015,01_tcp_ar,t002,/home/nis/Git/tuh-eeg-seizure-detection/data/r...,4,256.0288,286.0288,seiz
3,train,aaaaates,s007_2015,01_tcp_ar,t002,/home/nis/Git/tuh-eeg-seizure-detection/data/r...,4,276.0288,306.0288,seiz
4,train,aaaaates,s007_2015,01_tcp_ar,t002,/home/nis/Git/tuh-eeg-seizure-detection/data/r...,4,296.0288,326.0288,seiz
...,...,...,...,...,...,...,...,...,...,...
982,train,aaaaates,s002_2015,01_tcp_ar,t008,/home/nis/Git/tuh-eeg-seizure-detection/data/r...,0,180.0000,210.0000,bckg
983,train,aaaaates,s002_2015,01_tcp_ar,t008,/home/nis/Git/tuh-eeg-seizure-detection/data/r...,0,200.0000,230.0000,bckg
984,train,aaaaates,s002_2015,01_tcp_ar,t008,/home/nis/Git/tuh-eeg-seizure-detection/data/r...,0,220.0000,250.0000,bckg
985,train,aaaaates,s002_2015,01_tcp_ar,t008,/home/nis/Git/tuh-eeg-seizure-detection/data/r...,0,240.0000,270.0000,bckg


### feature extraction

In [6]:
def calc_coeffs_features(coeffs):
    mean = np.mean(coeffs)
    median = np.median(coeffs)
    std = np.std(coeffs)
    variance = np.var(coeffs)
    skew = sp.stats.skew(coeffs)
    kurtosis = sp.stats.kurtosis(coeffs)
    rms = np.sqrt(np.mean(coeffs ** 2))
    
    return {
        "mean": mean,
        "median": median,
        "variance": variance,
        "std": std,
        "skew": skew,
        "kurtosis": kurtosis,
        "rms": rms
    }
    
def extract_wavelet_features(channel_data: np.ndarray ) -> dict[str, float]:
    a5, d5, d4, d3, d2, d1 = pywt.wavedec(channel_data, 'db4', level=5)
    
    wavelet_features = {f"{coeff}_{stat}": value 
                        for coeff, data in zip(["a5", "d5", "d4", "d3"], [a5, d5, d4, d3])
                        for stat, value in calc_coeffs_features(data).items()}
    
    return wavelet_features

In [7]:
def extract_band_power(channel_data, sfreq=SAMPLING_FREQ, n_fft=256) -> dict[str, float]:
    frequency_bands = {
        "delta": (0.5, 4),
        "theta": (4, 7),
        "alpha": (7, 12),
        "beta": (12, 30),
        "gamma": (30, 50)
    }

    band_powers = {}

    n_fft = min(n_fft, channel_data.shape[-1])
    psds, freqs = mne.time_frequency.psd_array_welch(channel_data, sfreq=sfreq, n_fft=n_fft, fmin=0.5, fmax=50)

    # Calculate power within each frequency band
    for band, (fmin, fmax) in frequency_bands.items():
        # Find indices of frequencies within the band
        band_indices = np.where((freqs >= fmin) & (freqs <= fmax))[0]

        # Sum the power spectral density values within the band
        band_power = np.sum(psds[band_indices])

        band_powers[band] = band_power

    return band_powers 


In [8]:
def calc_power_ratios(band_powers : dict[str, float]) -> dict[str, float]:
    alpha_beta_ratio = band_powers["alpha"] / band_powers["beta"] if band_powers["beta"] != 0 else np.nan
    theta_beta_ratio = band_powers["theta"] / band_powers["beta"] if band_powers["beta"] != 0 else np.nan
    theta_alpha_beta_ratio = (band_powers["theta"] + band_powers["alpha"]) / band_powers["beta"] if band_powers["beta"] != 0 else np.nan
    theta_alpha_beta_alpha_ratio = (band_powers["theta"] + band_powers["alpha"]) / (band_powers["beta"] + band_powers["alpha"]) if (band_powers["beta"] + band_powers["alpha"]) != 0 else np.nan
    alpha_theta_ratio = band_powers["alpha"] / band_powers["theta"] if band_powers["theta"] != 0 else np.nan
    theta_alpha_ratio = band_powers["theta"] / band_powers["alpha"] if band_powers["alpha"] != 0 else np.nan
    
    return {
        "alpha_beta_ratio": alpha_beta_ratio,
        "theta_beta_ratio": theta_beta_ratio,
        "theta_alpha_beta_ratio": theta_alpha_beta_ratio,
        "theta_alpha_beta_alpha_ratio": theta_alpha_beta_alpha_ratio,
        "alpha_theta_ratio": alpha_theta_ratio,
        "theta_alpha_ratio": theta_alpha_ratio
    }

In [9]:
def calc_avg_band_powers(band_powers):
    avg_band_powers = {}
    for band in band_powers[0].keys():
        avg_band_powers[band] = np.mean([bp[band] for bp in band_powers])
        
    return avg_band_powers

In [10]:
def calc_asymmetry(band_powers):
    left_power = 0
    right_power = 0
    
    for i, channel in enumerate(CHANNELS):
        if channel in LEFT_HEMISPHERE:
            powers = list(band_powers[i].values())
            for power in powers:
                left_power += power
        elif channel in RIGHT_HEMISPHERE:
            powers = list(band_powers[i].values())
            for power in powers:
                right_power += power
            
    left_power = np.log(left_power) if left_power != 0 else 0
    right_power = np.log(right_power) if right_power != 0 else 0
    
    asymmetry = left_power - right_power
    return asymmetry

### preprocessing

In [11]:
def remove_powerline_noise(raw):
    powerline_noises = [50, 60]

    for freq in powerline_noises:
        raw.notch_filter(freqs=freq)

    return raw

In [12]:
def butterworth_filter(raw):
    iir_params = dict(order=4, ftype='butter')
    raw.filter(0.5, 50, method='iir', iir_params=iir_params)
    return raw

In [13]:
def min_max_normalization(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [14]:
def crop_raw_event(raw, start, stop):
    """Crops the raw data based on the onset and duration, handling edge cases."""
    if stop > raw.times[-1]:
        if stop - 1 / raw.info["sfreq"] == raw.times[-1]:
            return raw.copy().crop(start, raw.times[-1], include_tmax=True), True
        else:
            return None, False
    else:
        return raw.copy().crop(start, stop, include_tmax=False), True

### process windows

In [18]:
def process_windows(windows: pd.DataFrame):
    corrupted = []
    features = []

    windows = windows.groupby(["set", "patient_id", "session_id", "recording_id", "recording_path", "event_index"]).agg({
        "start": "min",
        "stop": "max",
        "label": "first"
    }).reset_index()
    
    recordings = windows.groupby("recording_path")
    
    for recording_path, events in recordings:
        raw = mne.io.read_raw_edf(recording_path, preload=True).pick(picks=CHANNELS)
        raw.set_meas_date(None)
        
        for _, event in events.iterrows():
            patient_id, session_id, recording_id, event_index, start, stop, label  = event.loc[["patient_id", "session_id", "recording_id", "event_index", "start", "stop", "label"]]

            raw_event, success = crop_raw_event(raw, start, stop)
            if not success:
                print("Corrupted annotation", patient_id, session_id, recording_id)
                corrupted.append((patient_id, session_id, recording_id))
            
            raw_event.resample(SAMPLING_FREQ)

            epochs = mne.make_fixed_length_epochs(raw_event, duration=WINDOW_LENGTH, overlap=OVERLAP, preload=True)
        
            raw_event = butterworth_filter(raw_event) 
            raw_event = remove_powerline_noise(raw_event)
            
            for epoch in epochs:
                channels = epochs.info["ch_names"]
                
                epoch_features = {}
                band_powers = []
                
                for j, channel in enumerate(channels):
                    channel_data = epoch[j]
                    channel_data = min_max_normalization(channel_data)
                    
                    wavelet_features = extract_wavelet_features(channel_data)
                    band_power = extract_band_power(channel_data)
                    
                    for key, value in wavelet_features.items():
                        epoch_features[f"{channel}_{key}"] = value
                    
                    band_powers.append(band_power)
                
                avg_band_powers = calc_avg_band_powers(band_powers)
                power_ratios = calc_power_ratios(avg_band_powers)
                
                asymmetry = calc_asymmetry(band_powers) 
                
                epoch_features.update({
                    "patient_id": patient_id,
                    "session_id": session_id,
                    "recording_id": recording_id,
                    "recording_path": recording_path,
                    "event_index": event_index,
                    "asymmetry": asymmetry,
                    "label": label
                })

                epoch_features = {**epoch_features, **avg_band_powers, **power_ratios}

                features.append(epoch_features)
                
            raw_event.close()
            
        raw.close()
        
    return pd.DataFrame(features), corrupted

In [23]:
features, corrupted = process_windows(windows)